<a href="https://colab.research.google.com/github/Seowoo-Jung/ESAA/blob/main/7%EC%9E%A5_%EC%95%99%EC%83%81%EB%B8%94%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. 앙상블 학습과 랜덤 포레스트

- 앙상블 기법(배깅, 부스팅, 스태깅), 랜덤 포레스트 

## 7.1 투표 기반 분류기

- 직접 투표 분류기: 다수결 투표로 정해지는 분류기

--> 강한 학습기가 된다.(약한 학습기가 충분히 많고 다양하면 앙상블이 강한 학습기가 된다.) 큰수의 법칙 때문임!(동전을 더 많이 던질수록 참값에 가까워진다.)
그러나 이런 가정은 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야하지만 같은 데이터로 학습하기 때문에 불가능하다. 그러면 앙상블의 오차가 낮아진다.

- 다른종류의 오차를 만들기 위해 각기 다른 알고리즘으로 학습시킨다.

In [ ]:
# 데이터셋
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y=make_moons(n_samples=1000, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting="hard"
) # 투표 기반 분류기
# estimators: 여러개의 모델을 tuple 형식으로 나열(이름, 객체)
# voting="hard"는 default값. 대다수의 모델이 예측하는 값으로 선택
# voting="soft"는 확률이 높은 투표에 더 비중을 둔다.(직접투표보다 성능이 더 좋다.) m
# model에 따라 확률을 연산하지 못할 수도 있는데 그때는 매개변수를 바꿔준다.(SVC에서 probability=True로 지정)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [ ]:
# 테스트셋의 정확도
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


LogisticRegression 0.87
RandomForestClassifier 0.995
SVC 0.99
VotingClassifier 0.995


투표 기반의 분류기의 성능이 높다!

In [ ]:
## 간접 투표방식: 이 경우 직접 투표방식과 비슷
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC(probability=True) 

voting_clf=VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting="soft"
) # 투표 기반 분류기

voting_clf.fit(X_train, y_train)

# 정확도 판단
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.87
RandomForestClassifier 0.995
SVC 0.99
VotingClassifier 0.995


## 7.2 배깅과 페이스팅

- 모델의 오차 독립성 증가를 위해 훈련세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습한다.

- 배깅 (boostrap aggregating): 훈련세트에서 중복을 허용하여 샘플링하는 반식

- 페이스팅:  중복을 허용하지 않고 샘플링하는 방식

--> 같은 훈련 샘플을 여러개의 예측기에 걸쳐 사용하지만 배깅만 한 예측기를 위해 훈련샘플을 여러번 샘플링 할 수 있다. 훈련을하고나서 앙상블은 예측을 모아 새로운 샘플에 대한 예측을 만든다.

- 수집함수: 분류-> 통계적 최빈값, 회귀-> 평균 계산 

- 배깅의 효과: 개별 예측기에 비해 편향은 비슷하지만 분산이 줄어든다.

- 배깅과 페이스팅은 CPU 병렬적 수행 가능.

#### 7.2.1 사이킷런의 배깅와 페이스팅

- 만약 모델이 분류될 확률을 추정할 수 있으면 자동으로 간접투표방식을 사용한다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, 
                          max_samples=100, bootstrap=True, n_jobs=-1) # DecisionTreeClassifier 500개로 학습
                          # sample은 100개 씩 뽑고 boostrap=True(배깅), False(페이스팅)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)

- 부스트랩은 예측기가 학습하는 서브셋에 다양성을 증가시키므로 편향이 높지만 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킨다.

#### 7.2.2 obb 평가

- 배깅을 사용하면 어떤건 여러번 선택되지만 어떤건 선택이 안될수도있다.(대략 37%정도는 아예 선택이 되지 않음: oob 샘플)

- oob 샘플은 훈련에 사용되지 않으므로 test data로 사용하여 앙상블의 평가를 평균하여 얻을 수 있다.

- BaggingClassifier에서 oob_score=True로 지정하면 oob_score_로 평가결과를 알 수 있다.

In [ ]:
bag_clf=BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                          bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_ # 테스트 데이터에서 98%정도의 정확도를 얻을 것이라고 예측

0.9875

In [ ]:
# 실제 정확도 구해보기: oob 이용한 것과 거의 비슷
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.99

In [ ]:
# oob 샘플에 대한 결정함수의 값을 구할 수 있음 (단, 기반이 되는 예측기가 확률을 모델링 할 수 있어야함)
bag_clf.oob_decision_function_ # 각각 i번째 sample이 0 또는 1에 속할 확률

array([[0.        , 1.        ],
       [1.        , 0.        ],
       [0.02162162, 0.97837838],
       ...,
       [1.        , 0.        ],
       [0.03351955, 0.96648045],
       [0.        , 1.        ]])

## 7.3 랜덤 패치와 랜덤 서브스페이스

* BaggingClassifier의 샘플링 옵션: max_features, bootstrap_features 조절

--> 샘플링이 아니라 특서에 대한 샘플링 ( 무작위로 선택한 입력 특성의 일부로 훈련한다.) 이미지와 같이 고차원 데이터셋을 다룰 때 사용

- 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링하는 것

- 랜덤 서브스페이스 방식: 훈련샘플을 모두 사용하고(bootstrap=False 이고 max_samples=1.0) 특성은 샘플링하는 것(bootstrap_features=True, max_features=1.0보다 작게)

--> 다양한 예측기를 만들어 편향을 늘리는 대신 분산을 낮춘다.

cf) max_...에 정수가 오면 그 숫자만큼 샘플링 하는거고 실수가 오면 전체 sample에 대한 특정 비율만큼 샘플링하는 것



#### 7.4 랜덤 포레스트

- 배깅방법을 이용한 결정 트리 앙상블. (max_samples로 훈련세트의 크기 결정)

- RandomForestClassifier = BaggingClassifier에 DecisionTreeClassifier 넣는것과 같음 따라서 두 객체의 파라미터 전부를 가질 수 있음.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1) # 트리 500개로 이루어지고 최대 16개의 리프노드 가진다.
rnd_clf.fit(X_train, y_train)

y_pred_rf=rnd_clf.predict(X_test)

- 트리노드를 분할할 때, 전체 특성중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적을 찾는 방식으로 해본다.( 무작위성이 증가하여 편향이 늘어나는대신 분산을 낮춰 더 모델이 좋아진다.)

In [ ]:
bag_clf=BaggingClassifier(DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16), n_estimators=500,
                                                 max_samples=1.0, bootstrap=True, n_jobs=-1)

#### 7.4.1 엑스트라 트리

- 가장 최적이 되는 특성을 찾아 트리를 만드는 것이 아니라 무작위 특성의 서브셋을 만들어 분할에 사용한다.

- 트리를 무작위하게 만들기 위해 임계값을 찾는대신 후보특성을 사용해 무작위로 분할하고 그 중 최상의 분할을 선택한다.(편향은 손해보지만 분산은 낮아진다.)

- 무작위성을 기반으로 하므로 랜덤 포레스트보다 더 빠르다.

#### 7.4.2 특성 중요도

- 랜덤 포레스트는 특성 중요도를 판단하기 쉽다.: 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성중요도를 측정한다.(각 노드의 가중치는 연관된 훈련 샘플 수와 같다.)

중요도: 각 결정트리의 특성중요도를 모두 더한 후 트리수로 나눈것.

각 특성의 중요도: 현재 노드의 샘플 비율*불순도 - 왼쪽 자식 노드의 샘플 비율*불순도- 오른쪽 자식 노드의 샘플 비율*불순도 --> 특성중요도합이 1이되도록 전체 합으로 나누어 정규화한다. 

* 샘플 비율: 트리 전체 샘플수에 대한 비율

In [ ]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.1065264794932518
sepal width (cm) 0.02572589387223662
petal length (cm) 0.42435564221832883
petal width (cm) 0.4433919844161827


- 꽃잎의 길이와 너비는 꽃밭침의 길이와 너비보다 더 중요하다.

- 변수 중요도는 어떤 특성이 중요한지 빠르게 확인할 수 있어 편리하다.

## 7.5 부스팅 (가설 부스팅)

- 약한 학습기(예측 성능이 낮은거)를 여러개 연결하여 강한 학습기(예측 성능이 높은거)를 만드는 앙상블 기법--> 대체로 이전 모델에 대한 보완

- 에이다부스트, 그레디언트 부스팅

#### 7.5.1 에이다 부스트

- 모델이 과소적합했던 훈련 샘플의 가중치를 높이는 것: 새로운 예측기는 학습하기 어려운 샘플에 점점 맞춰지게된다.

--> 즉 먼저 훈련을 시키고 알고리즘이 오분류한 훈련샘플에 가중치를 높여 (가중치 업데이트) 다시 훈련하여 예측한다. ---반복

- 연속된 학습 기법을 사용하기 때문에 학습이 병렬화 될 수 없다. ( 확장성이 높지 않다.)

- 처음에 샘플 가중치를 $\frac{1}{m}$으로 맞추고 에러율을 계산하여 가중치를 업데이트 한다. 예측기가 정확할수록 가중치가 더 높고 무작위로 예측하는 것은 가중치가 0이다. 그보다 나쁘면 가중치가 음수가 된다. 

- 예측을 할 때, 모든 예측기의 예측을 계산하고 각각의 가중치를 더해 예측결과를 만든다. 따라서 가중치 합이 가장 큰 클래스가 에측 결과가 된다. 

사이킷런의 SAMME를 사용하고 확률을 추정하기 위해서는 SAMME.R을 이용한다.

- AdaBoostClassifier를 이용하면 200개의 아주 얕은 결정트리를 기반으로 한다.(max_depth=1)

*참고 블로그: https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-14-AdaBoost


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200,
                           algorithm="SAMME.R", learning_rate=0.5)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

#### 7.5.2 그레디언트 부스팅

- 이전의 오차를 보정하도록 예측기를 순차적으로 추가하지만 가중치를 수정하지 않고 이전 예측기의 잔여오차에 대해 새로운 예측기를 학습시킨다.



In [ ]:
## GBRT 구현
from sklearn.tree import DecisionTreeRegressor

# 1차 학습 : y값 자체에 대한 학습
tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

# 2차 학습: 1차 모형의 오차에 대한 학습으로 첫번째 모델과 오차를 더해주면 더 정확한 모델이 나온다.
# 앙상블 예측값이 1,2차모형 예측값의 합과 같다.
y2=y-tree_reg1.predict(X) # 오차
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

# 3차 학습 : 2차 모형에서의 오차에 대한 학습으로 1,2,3,차 모델을 전부 더해주면 더 정확한 모델이 나온다.
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
## 3개의 트리를 포함하는 앙상블 모델 형성. 
## 새로운 샘플에 대한 예측: 모든 트리의 예측을 더한다.
y_pred=sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

- GradientBoostingRegressor를 활용하면 앙상블 구현가능. n_estimators= 트리수로 앙상블의 훈련을 제어하는 매개변수와 RandomForestRegressor와 같이 결정트리를 제어하는 변수도 갖는다.

In [ ]:
from skelarn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0) # 트리수 3개 만들고 각 트리의 깊이는 2, 학습률은 1로한다.
gbrt.fit(X,y)

- learning_rate는 각 트리의 기여 정도를 조절. 낮게 설정하면 트리가 많이 필요하지만 예측 성능은 좋아짐(축소)
 
 But 과대적합 될 수 있으므로 조기종료기법을 사용한다. (stages_predict() 사용한다.-> 각 stage별 예측값을 반환)

 - 120개의 트리로 GBRT 앙상블을 훈련시키고 검증오차가 최소가되는 트리의 개수 구한다.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120) # 120개의 tree만든다.
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred)
for y_pred in gbrt.staged_predict(X_val)] # 각 트리마다 mse를 계산해서 list에 집어넣는다.
bst_n_estimators=np.argmin(errors)+1 # error값이 최소가 되는 index 반환하여 +1하면 stage가 나온다.

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators) # 최적의 모형으로 다시 fit
gbrt_best.fit(X_train, y_train)

- 조기 종료 구현: 5번 연속으로 검증오차가 향상되지 않으면 훈련을 멈춥니다.

In [ ]:
gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True) # warm_start=True옵션을 설정하면 fit이 호출될 때 기존의 tree에서 훈련을 추가해준다.

min_val_error=float('inf')
error_going_up=0

for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators # 트리개수 바뀜
  gbrt.fit(X_train, y_train)
  y_pred=gbrt.predict(X_val)
  val_error=mean_squared_error(y_val, y_pred)

  if val_error < min_val_error:
    min_val_error=val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up ==5:
      break

- 훈련 시 훈련 샘플의 비율을 지정할 수 있는 subsample매개변수도 지원한다. 

- 무작위로 일정비율의 훈련샘플만 학습단다. --> 편향이 높아지지만 분산이 낮아진다. > 확률적 그레이디언트 부스팅

- xgboost를 사용하면 더 빠르게 편하게 구현할 수 있음 (extreme gradient boosting)

In [ ]:
impport xgboost

xgb_reg=sgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred=xgb_reg.predict(X_val)

In [ ]:
# 조기 종료도 가능
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

## 7.6 스태킹

- 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬수는 없을까?

- 각기 다른 예측값을 블렌더(메타학습기)가  최종예측을 만들어 준다.

- 훈련세트를 두 개로 나누어 첫번째 서브셋은 첫번째 레이어의 예측을 한다. 그다음 두번째 세트에 대한 예측을 한다. 

- 여러가지 모델로 학습을 시키고 예측값들을 데이터로 넣어 최종 분류기 모델에 학습을 시킨다.

* 참고 블로그: https://huidea.tistory.com/35